In [3]:
"""
2-input XOR example -- this is most likely the simplest possible example.
"""

from __future__ import print_function
import os
import neat
import visualize
import csv


# Date, Open, High, Low, Close, Adj Closec,Volume
input_training_data = []



def decide(res):
    if (res > 0.6):
        return 1.0
    elif (res < 0.4):
        return -1.0
    else:
        return 0.0


def load_data_from_csv(filename):
    data = []
    with open(filename, newline='') as File:  
        reader = csv.reader(File)
        for row in reader:
            data.append((round(float(row[1]),2),round(float(row[2]),2),round(float(row[3]),2),round(float(row[4]),2),round(float(row[5]),2),round(float(row[6]),2)))
    return data
            
input_training_data = load_data_from_csv('CAC40_2005_2017.csv')

#for r in input_training_data:
#    print(r)


    
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        money = 100000.0
        genome.fitness = money
        shares = 0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        for input_data in input_training_data:
            output = net.activate(input_data)
            decision = decide(output[0])
            if (decision == -1 and shares > 0):
                shares = shares-1
                money = money + input_data[4]
                
            elif (decision == 1 and money > input_data[4]):
                shares = shares+1
                money = money - input_data[4]
                
            genome.fitness = money + shares*input_data[4]



def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5))

    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 350)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))

    # Show output of the most fit genome against training data.
    print('\nOutput:')
    winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
    #for xi, xo in zip(xor_inputs, xor_outputs):
    #    output = winner_net.activate(xi)
    #    print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))

    node_names = {-1:'Open', -2: 'High',-3: 'Low',-4: 'Close',-5: 'Adj Closec',-5: 'Volume', 0:'decide'}
    visualize.draw_net(config, winner, True, node_names=node_names)
    visualize.plot_stats(stats, ylog=False, view=True)
    visualize.plot_species(stats, view=True)

    p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
    p.run(eval_genomes, 10)


if __name__ == '__main__':
    # Determine path to configuration file. This path manipulation is
    # here so that the script will run successfully regardless of the
    # current working directory.
    local_dir = os.path.dirname('/home/mrrob0t/projects/AI/')
    config_path = os.path.join(local_dir, 'config-feedforward')
    run(config_path)

    

IndexError: list index out of range